## 다중 회귀
- ax + by + cz + d = 0
- 서로 다른 특성을 여러개 사용, 평면 학습

### 특성 공학
- 기존의 특성을 사용하여 새로운 특성을 뽑아냄 ex) 농어의 길이 * 농어의 높이

In [57]:
import pandas as pd

df = pd.read_csv('data/Fish.csv')
perch_df = df.loc[df['Species'] == 'Perch']
perch_df.head()
# 농어의 길이, 높이, 두께 사용

,Species,Weight,Length1,Length2,Length3,Height,Width
72,Perch,5.9,7.5,8.4,8.8,2.1120,1.4080
73,Perch,32.0,12.5,13.7,14.7,3.5280,1.9992
74,Perch,40.0,13.8,15.0,16.0,3.8240,2.4320
75,Perch,51.5,15.0,16.2,17.2,4.5924,2.6316
76,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415


In [58]:
perch_full = perch_df[['Length2', 'Height', 'Width']]
perch_weight = perch_df[['Weight']]

In [59]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight) 
# 책처럼 정확히 하려면 random_state = 42를 옵션으로 넣어주면 됨

In [60]:
# PolynomialFeatures() : 특성을 주면 특성으로 만들 수 있는 조합을 만들어줌(곱, 제곱 등)
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False)
# include_bias=False : 1을 빼고 변환 => 선형 방정식의 절편을 항상 값이 1인 특성과 곱해지는 계수여서 1도 나옴
poly.fit([[3, 5]]) # 특성 조합 찾기
poly.transform([[3, 5]]) # 실제로 데이터 변환

array([[ 3.,  5.,  9., 15., 25.]])

In [61]:
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
train_poly.shape

(42, 9)

In [62]:
poly.get_feature_names_out() # 어떤 조합을 만들어줬는지 알려줌

array(['Length2', 'Height', 'Width', 'Length2^2', 'Length2 Height',
       'Length2 Width', 'Height^2', 'Height Width', 'Width^2'],
      dtype=object)

In [63]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_poly, train_target)
lr.score(train_poly, train_target)

0.994961429366647

In [64]:
test_poly = poly.transform(test_input)

In [65]:
lr.score(test_poly, test_target)

0.9408207565632757

a * 길이 + b * 높이 + c * 너비 + d * 길이^2 + e * 높이 * 길이 => 특성공학

In [66]:
poly = PolynomialFeatures(degree=5, include_bias=False) # 5제곱까지 만들어줘 ex) 높이^5
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
train_poly.shape

(42, 55)

In [67]:
lr.fit(train_poly, train_target)

LinearRegression()

In [68]:
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

0.999999999999986
-126.05483828986306


## 규제
- 훈련 세트를 너무 과도하게 학습하지 못하도록 훼방하는 것 => 과대적합이 되지 않도록 만드는 것
- 선형 회귀모델의 경우 특성에 곱해지는 계수(기울기)의 크기를 작게 만드는 일

In [69]:
# 정규화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_poly)

train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

In [70]:
train_poly[:5]

array([[3.50000000e+01, 1.14884000e+01, 7.79570000e+00, 1.22500000e+03,
        4.02094000e+02, 2.72849500e+02, 1.31983335e+02, 8.95601199e+01,
        6.07729385e+01, 4.28750000e+04, 1.40732900e+04, 9.54973250e+03,
        4.61941671e+03, 3.13460420e+03, 2.12705285e+03, 1.51627734e+03,
        1.02890248e+03, 6.98183827e+02, 4.73767597e+02, 1.50062500e+06,
        4.92565150e+05, 3.34240638e+05, 1.61679585e+05, 1.09711147e+05,
        7.44468497e+04, 5.30697069e+04, 3.60115868e+04, 2.44364339e+04,
        1.65818659e+04, 1.74196006e+04, 1.18204433e+04, 8.02101507e+03,
        5.44283166e+03, 3.69335005e+03, 5.25218750e+07, 1.72397802e+07,
        1.16984223e+07, 5.65878547e+06, 3.83989014e+06, 2.60563974e+06,
        1.85743974e+06, 1.26040554e+06, 8.55275188e+05, 5.80365306e+05,
        6.09686021e+05, 4.13715514e+05, 2.80735528e+05, 1.90499108e+05,
        1.29267252e+05, 2.00123340e+05, 1.35797980e+05, 9.21486296e+04,
        6.25294272e+04, 4.24306827e+04, 2.87922490e+04],
       

### 릿지(Ridge)
- 계수를 제곱한 값을 기준으로 규제 적용, L2규제

In [71]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)

print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

0.9950265092854239
0.9672774000918201


### 라쏘(Lasso)
- 라쏘 : 계수의 절댓값을 기준으로 규제 적용, L1규제
- 0을 만들 수 있음

In [72]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

0.9913897915133566
0.9724208449655563


C:\Users\1-13\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.785e+03, tolerance: 5.289e+02
  model = cd_fast.enet_coordinate_descent(


In [73]:
lasso.coef_

array([  0.        ,   0.        ,   0.        ,  50.6365306 ,
         0.        ,   0.        ,  13.59471465,   0.        ,
        52.1910603 ,  25.43834016,   6.93528342,   0.        ,
        80.84116759,   0.        ,   0.        , 125.17395285,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  -0.        ,  -0.        ,
        -0.        ,   0.        ,  -0.        ,  -0.        ,
         0.        ,  -0.        ,  -0.        ,  -0.        ,
         0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,   0.        ,   0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ])